In [1]:
import sys
import os
work_path = os.path.abspath(os.path.join(os.getcwd(), '../../../../'))
sys.path.append(work_path)
from helper.regrassion import *
from helper.plot import *
from helper.util import *
from helper.analysis import *

from sklearn.linear_model import LinearRegression

from sklearn.model_selection import cross_val_score, cross_validate, cross_val_predict

In [2]:
origin = my_read_excel('C:/Users/Jihwan/Desktop/01Class/E.추론통계/E.InferentialStatistics/수업자료/fish.xlsx')
my_pretty_table(origin.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56 entries, 0 to 55
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   길이      56 non-null     float64
 1   높이      56 non-null     float64
 2   두께      56 non-null     float64
 3   무게      56 non-null     float64
dtypes: float64(4)
memory usage: 1.9 KB
None

데이터프레임 상위 5개 행
+----+--------+--------+--------+--------+
|    |   길이 |   높이 |   두께 |   무게 |
|----+--------+--------+--------+--------|
|  0 |    8.4 |   2.11 |   1.41 |    5.9 |
|  1 |   13.7 |   3.53 |      2 |     32 |
|  2 |     15 |   3.82 |   2.43 |     40 |
|  3 |   16.2 |   4.59 |   2.63 |   51.5 |
|  4 |   17.4 |   4.59 |   2.94 |     70 |
+----+--------+--------+--------+--------+

데이터프레임 하위 5개 행
+----+--------+--------+--------+--------+
|    |   길이 |   높이 |   두께 |   무게 |
|----+--------+--------+--------+--------|
| 51 |     42 |   12.8 |   6.87 |   1100 |
| 52 |     43 |  11.93 |   7.28 |   1000 |
| 53 |     43

In [3]:
yname = '무게'
x = origin.drop(yname, axis=1)
y = origin[yname]
x.shape, y.shape

((56, 3), (56,))

## #04. K-Fold 교차 검증을 적용한 회귀 모델

### [1] 한가지 평가지표만 확인하기

1개의 폴드부터 10개의 폴드 까지 반복적으로 수행

In [4]:
k = 10

model = LinearRegression(n_jobs=-1)

scores = cross_val_score(model, x, y, cv=k, scoring='r2', n_jobs=-1)

print('R2:', scores)
print('R2 최대값:', scores.max())
print('R2 최대값의 Fold수:', scores.argmax()+1)

R2: [-5.80130229e+01 -2.31476956e+00 -4.52072266e+01 -4.47723355e+00
 -5.35954360e+00 -1.75520025e+01  6.25492054e-01  5.30196516e-01
 -5.03267727e-02 -1.75845285e+01]
R2 최대값: 0.62549205419385
R2 최대값의 Fold수: 7


### [2] 전체 정보 얻기

#### (1) K-Fold 수행

리턴받고자 하는 정보를 bool 형식으로 설정

In [6]:
model = LinearRegression(n_jobs=-1)
result = cross_validate(model, x,y, cv=k,
                        return_train_score=True, return_estimator=True,
                        n_jobs=-1)
result_df = DataFrame(result)
my_pretty_table(result_df)

+----+------------+--------------+-----------------------------+--------------+---------------+
|    |   fit_time |   score_time | estimator                   |   test_score |   train_score |
|----+------------+--------------+-----------------------------+--------------+---------------|
|  0 | 0.00256085 |   0.00199127 | LinearRegression(n_jobs=-1) |      -58.013 |      0.969328 |
|  1 | 0.00256085 |   0.00199127 | LinearRegression(n_jobs=-1) |     -2.31477 |       0.93761 |
|  2 | 0.00506997 |    0.0013845 | LinearRegression(n_jobs=-1) |     -45.2072 |      0.939373 |
|  3 | 0.00299358 |   0.00207639 | LinearRegression(n_jobs=-1) |     -4.47723 |      0.940413 |
|  4 |  0.0024631 |  0.000996828 | LinearRegression(n_jobs=-1) |     -5.35954 |      0.943642 |
|  5 | 0.00175548 |   0.00102329 | LinearRegression(n_jobs=-1) |      -17.552 |      0.948462 |
|  6 | 0.00199366 |  0.000997066 | LinearRegression(n_jobs=-1) |     0.625492 |      0.945635 |
|  7 | 0.00199389 |   0.00118351 | Linea

#### (2) 최적의 성능을 갖는 폴드에 대한 학습 객체 추출

In [7]:
maxindex = result_df['test_score'].argmax()
best_model = result_df.iloc[maxindex]['estimator']
best_model

LinearRegression(n_jobs=-1)

### [3] 최적의 k폴드 수행에 대한 추정치 얻기

In [8]:
y_predict = cross_val_predict(model, x, y, cv=maxindex+1, n_jobs=-1)
y_predict

array([-415.07853167, -226.51199473, -170.5319201 , -102.50092462,
        -68.04456227,   -9.39059305,   -7.16198571,   16.88762632,
         63.61423719,   41.92835141,  139.71780356,  139.54040268,
        147.04399414,  187.58426947,  161.29346999,  134.95737725,
        171.75681189,  155.22220036,  160.06735997,  161.06534327,
        227.92268556,  168.52744392,  116.5759082 ,  199.41972277,
        276.25603007,  221.46559183,  269.52394449,  219.52200383,
        265.68650727,  303.05013887,  447.64446312,  319.07001843,
        312.17502048,  343.23703929,  353.00323619,  372.34359822,
        388.28133524,  641.13625561,  688.40093468,  869.19466922,
        759.52498486,  729.72075358,  728.31303342,  709.7854361 ,
        814.30380312,  744.33044163,  920.9310011 ,  866.73418149,
        754.01238957,  813.11997327,  655.80233646,  749.79587392,
        752.08029315,  795.94389038,  883.27908607,  810.40730325])